In [42]:
import os
import requests
import random
import glob
import os
import requests
import shutil
import random
from sklearn.model_selection import train_test_split
from pathlib import Path
import cv2


In [28]:
from yt_dlp import YoutubeDL

robot_videos = [ 'https://www.youtube.com/watch?v=gtJXGo8WEMY',
                'https://www.youtube.com/watch?v=hKLC0Vz1GmM ',
                'https://www.youtube.com/watch?v=6ldHWWHfeBc&t=5s',
                'https://www.youtube.com/watch?v=WJKc56uUuF8',
                'https://www.youtube.com/watch?v=RG205OwGdSg',
                'https://www.youtube.com/watch?v=yVdB_0ry53o',
                'https://www.youtube.com/watch?v=TWNvSHpMrSM',
                'https://www.youtube.com/watch?v=UsmBD2_3FH8',
                'https://www.youtube.com/watch?v=WGKo_6IkFBY',
                'https://www.youtube.com/watch?v=G6xE7uWt6Fo&t=1s',
                'https://www.youtube.com/watch?v=DrNcXgoFv20',
                'https://www.youtube.com/watch?v=cpraXaw7dyc',
                'https://www.youtube.com/watch?v=raYWbqbZbmc',
                'https://www.youtube.com/watch?v=F_7IPm7f1vI]']

root = os.getcwd()

# Directories for youtube videos
dataset_dir = os.path.join(root,'dataset')
save_path = os.path.join(dataset_dir,'youtube_videos/')
os.makedirs(save_path, exist_ok=True)


# # Directories for dataset
# os.makedirs(dataset_dir+'/robot/train', exist_ok=True)
# os.makedirs(dataset_dir+'/robot/test', exist_ok=True)
# os.makedirs(dataset_dir+'/person/train', exist_ok=True)
# os.makedirs(dataset_dir+'/person/test', exist_ok=True)


In [40]:

# Function to download frames from YouTube videos
def youtube_downloader(urls,save_path):
    ydl_opts = {  # Youtube downloader dictionaty options.
        'outtmpl': save_path+'%(title)s.%(ext)s',  # Save path+video title+file extension
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',  # Prefer MP4
    }
    for url in urls:
        with YoutubeDL(ydl_opts) as ydl:
            try:
                ydl.download([url])
                print("Download complete!")
            except Exception as e:
                print(f"Error downloading video {url}: {e}")

In [ ]:
youtube_downloader(urls= robot_videos, save_path= save_path)

In [39]:
def extract_frames(video_path, output_dir, num_frames , frames_collected):
    '''
    Extracts random number of frames from a given video file and saves 
    the resulting frames
    '''

    # Open video and get information
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Generating random frames to extract 
    random_frames = random.sample(range(0, total_frames), num_frames)
    
    # Extract frames
    for i, frame_idx in enumerate(random_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            # Save frame
            frames_collected += 1
            save_path = os.path.join(output_dir, f"robot{frames_collected}.png")
            cv2.imwrite(save_path, frame)
            
            # # Break if collected enough frames
            # if frames_collected >= num_frames:
            #     break

    # Release video
    cap.release()
    
    return frames_collected


In [43]:
videos = glob.glob(save_path+'*.mp4', recursive= True)
output_train = os.path.join(dataset_dir,'train','robot')
output_test = os.path.join(dataset_dir,'test','robot')

frames_collected_train = 0
frames_collected_test = 0

for video in videos:
    # extracting num_frames number of random frames from each video 
    frames_collected_train = extract_frames(video_path= video, 
                                            output_dir= output_train, 
                                            num_frames= 25,
                                            frames_collected= frames_collected_train)
    
    frames_collected_test = extract_frames(video_path= video, 
                                           output_dir= output_test, 
                                           num_frames= 10,
                                           frames_collected= frames_collected_test)
    
print (" Robot images for train/test set extracted from youtube videos!")


 Robot images for train/test set extracted from youtube videos!


In [ ]:
# Configuration
NUM_IMAGES_PER_CLASS = 180
TRAIN_RATIO = 0.8
PEXELS_API_KEY = "Uf8MkiMbaC21S7H2AyZ6SS3urnALdpti85XPn7nxjnnNy7bfIe170fdl"  # Replace with your Pexels API key
TRAIN_DIR = os.path.join(dataset_dir, "train")
TEST_DIR = os.path.join(dataset_dir, "test")

# Create directory structure
def create_dirs():
    for dir_path in [os.path.join(TRAIN_DIR, "human"), os.path.join(TRAIN_DIR, "robot"),
                     os.path.join(TEST_DIR, "human"), os.path.join(TEST_DIR, "robot")]:
        os.makedirs(dir_path, exist_ok=True)

# Download images from Pexels for a given class
def download_images(class_name, search_terms):
    images = []
    images_per_term = NUM_IMAGES_PER_CLASS // len(search_terms)
    
    headers = {"Authorization": PEXELS_API_KEY}
    for term in search_terms:
        page = 1
        while len(images) < NUM_IMAGES_PER_CLASS:
            url = f"https://api.pexels.com/v1/search?query={term}&per_page={images_per_term}&page={page}"
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"Error fetching Pexels images for {term}: {response.status_code}")
                break
            
            data = response.json()
            photos = data.get("photos", [])
            if not photos:
                break
                
            for photo in photos:
                img_url = photo["src"]["medium"]
                img_name = f"{class_name}_{len(images)}_{term.replace(' ', '_')}.jpg"
                img_path = os.path.join(f"temp_{class_name}", img_name)
                os.makedirs(f"temp_{class_name}", exist_ok=True)
                
                # Download image
                img_response = requests.get(img_url, stream=True)
                if img_response.status_code == 200:
                    with open(img_path, "wb") as f:
                        img_response.raw.decode_content = True
                        shutil.copyfileobj(img_response.raw, f)
                    images.append(img_path)
                
                if len(images) >= NUM_IMAGES_PER_CLASS:
                    break
            page += 1
    
    return images[:NUM_IMAGES_PER_CLASS]

# Split and save images to train/test folders
def split_and_save_images(images, class_name):
    train_images, test_images = train_test_split(
        images, train_size=TRAIN_RATIO, random_state=42
    )
    
    # Save train images
    for idx, img_path in enumerate(train_images):
        dest_path = os.path.join(TRAIN_DIR, class_name, f"{class_name}_{idx}.jpg")
        shutil.copy(img_path, dest_path)
    
    # Save test images
    for idx, img_path in enumerate(test_images):
        dest_path = os.path.join(TEST_DIR, class_name, f"{class_name}_{idx}.jpg")
        shutil.copy(img_path, dest_path)

def main():
    # Create directory structure
    create_dirs()
    
    # Download human images
    print("Downloading human images...")
    human_search_terms = ["diverse people", "human portrait", "people in nature", "person"]  # Diverse human images
    human_images = download_images("human", human_search_terms)
    if len(human_images) < NUM_IMAGES_PER_CLASS:
        print(f"Warning: Only {len(human_images)} human images found.")

    # Split and save images
    print("Organizing images into train/test folders...")
    split_and_save_images(human_images, "human")
    # split_and_save_images(robot_images, "robot")
    
    # Clean up temporary folders
    for temp_dir in ["temp_human", "temp_robot"]:
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
    
    print(f"Dataset created at {dataset_dir} with {NUM_IMAGES_PER_CLASS} images per class.")

if __name__ == "__main__":
    main()